In [ ]:
import numpy as np
import os
import json
from pathlib import Path
from scipy.spatial.transform import Rotation
from scipy.linalg import expm
from plotly import graph_objects as go

from lac.utils.plotting import plot_path_3d, plot_3d_points
from lac.localization.imu_dynamics import propagate_state
from lac.util import load_data, skew_symmetric, pose_to_pos_rpy

%load_ext autoreload
%autoreload 2

Rollout trajectory using control inputs and/or wheel odometry measurements recorded and a simple dubins


In [ ]:
data_path = os.path.expanduser(
    "~/LunarAutonomyChallenge/output/LocalizationAgent/map1_preset0_4m_spiral/"
)
json_data = json.load(open(os.path.join(data_path, "data_log.json")))
initial_pose, lander_pose, pose_list, imu_data, cam_config = load_data(data_path)

In [ ]:
# define the ground truth
gt_translations = np.zeros((len(pose_list), 3))  # x, y, z
gt_rotations_euler = np.zeros((len(pose_list), 3))  # roll, pitch, yaw
dt = json_data["frames"][1]["mission_time"]  # assume consistent dt across all frames

for i in range(0, len(pose_list)):
    pos_i, rpy_i = pose_to_pos_rpy(pose_list[i])
    gt_translations[i, :3] = pos_i
    gt_rotations_euler[i] = rpy_i

In [ ]:
def dubins_step(x: np.ndarray, v: float, w: float, dt: float) -> np.ndarray:
    """
    Compute the next state of the Dubins car

    Args:
    x: state vector in world-frame (x, y, z, vx, vy, vz, roll, pitch, yaw)
    v: speed (linear)
    w: angular velocity (yaw) about robot z-axis
    dt: time

    Returns:
    x_next: next state vector in the world frame
    """

    # project the body frame velocities to the world frame
    rot_rover_to_world = Rotation.from_euler("xyz", x[6:]).as_matrix()
    v_world = rot_rover_to_world @ np.array([v, 0, 0])
    new_pose = rot_rover_to_world @ expm(skew_symmetric(np.array([0, 0, w])) * dt)

    # compute the next state
    x_next = np.zeros_like(x)
    x_next[:3] = x[:3] + v_world * dt
    x_next[3:6] = x[3:6]
    x_next[6:] = Rotation.as_euler(Rotation.from_matrix(new_pose), "xyz", degrees=False)

    return x_next

In [ ]:
def extract_controls(data_path: str):
    frame_data = json.load(open(os.path.join(data_path, "data_log.json")))["frames"]

    nom_controls = {}
    wheel_odom = {}
    for frame in frame_data:
        frame_id = frame["step"]
        nom_controls[frame_id] = np.array([frame["control"]["v"], frame["control"]["w"]])
        wheel_odom[frame_id] = np.array([frame["linear_speed"], frame["angular_speed"]])

    return nom_controls, wheel_odom

In [ ]:
# Run Dubins car simulation
n_sim = len(pose_list)
nom_controls, wheel_odom = extract_controls(data_path)

# initial state
init_pos, init_rpy = pose_to_pos_rpy(pose_list[0])
v0 = np.zeros(3)
x0 = np.hstack((init_pos, v0, init_rpy)).T

x_nom_traj = np.zeros((n_sim, 9))
x_nom_traj[0] = x0

x_wheel_traj = np.zeros((n_sim, 9))
x_wheel_traj[0] = x0

x_imu_traj = np.zeros((n_sim, 9))
x_imu_traj[0] = x0

for i in range(1, n_sim):
    v_nom, w_nom = nom_controls[i]
    v_wheel, w_wheel = wheel_odom[i]
    a_k = imu_data[i - 1][:3]  # imu data starts with step 1 at index 0
    omega_k = imu_data[i - 1][3:]

    x_nom_traj[i] = dubins_step(x_nom_traj[i - 1], v_nom, w_nom, dt)
    x_wheel_traj[i] = dubins_step(x_wheel_traj[i - 1], v_wheel, w_wheel, dt)
    x_imu_traj[i], _ = propagate_state(x_imu_traj[i - 1], a_k, omega_k, dt)

In [ ]:
# pllot
fig = plot_path_3d(
    gt_translations,
    color="blue",
    name="Ground truth",
)
fig = plot_path_3d(
    x_wheel_traj,
    fig=fig,
    color="green",
    name="Wheel odometry",
)
fig = plot_path_3d(
    x_nom_traj,
    fig=fig,
    color="red",
    name="Nominal controls",
)
# fig = plot_path_3d(
#     x_imu_traj,
#     fig=fig,
#     color="green",
#     name="IMU",
# )

fig.update_layout(height=700, width=1200, scene_aspectmode="data")
fig.show()


### Dynamics characterization


In [ ]:
import matplotlib.pyplot as plt

from lac.utils.plotting import plot_path_rover_frame
from lac.utils.frames import invert_transform_mat
from lac.util import get_positions_from_poses

In [ ]:
data_path = Path("../../output/DataCollectionAgent/map1_preset0_v0.2_w-0.5")
initial_pose, lander_pose, poses, imu_data, cam_config = load_data(data_path)

In [ ]:
# Transform poses from world to initial local rover frame
poses_local = [invert_transform_mat(initial_pose) @ pose for pose in poses]

positions_local = get_positions_from_poses(poses_local)
plot_path_rover_frame(positions_local, color="blue")

In [ ]:
json_data = json.load(open(f"{data_path}/data_log.json"))
N = len(json_data["frames"])
wheel_odoms = np.zeros((N, 2))
for i, frame in enumerate(json_data["frames"]):
    wheel_odoms[i] = [frame["linear_speed"], frame["angular_speed"]]

In [ ]:
plt.plot(wheel_odoms[:, 0], label="linear speed")

In [ ]:
plt.figure()
plt.plot(wheel_odoms[:, 1], label="angular speed")
plt.legend()
plt.show()